# Portfolio selection

Formulation: https://github.com/opvious/examples/blob/main/sources/portfolio-selection.md

In [1]:
%pip install 'opvious>=0.16.0' yfinance

## Download input data

We gather tickers from Wikipedia and recent performance data via `yfinance`.

In [2]:
import logging
import pandas as pd
import yfinance as yf

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s', datefmt='%H:%M:%S')

In [3]:
tickers_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [4]:
values_df = yf.download(tickers=list(tickers_df['Symbol'])[:10], start='2022-1-1', interval='1mo')['Adj Close']
returns_df = values_df.head(100).dropna(axis=1).pct_change().dropna(axis=0, how='all')
returns_df.head()

[*********************100%***********************]  10 of 10 completed


,AAP,ABBV,ABT,ACN,ADBE,ADM,ADP,AOS,ATVI,MMM
Date,,,,,,,,,,
2022-02-01,-0.116755,0.090681,-0.050326,-0.103911,-0.124686,0.046000,-0.008391,-0.099266,0.031515,-0.104626
2022-03-01,0.012128,0.097043,-0.018736,0.067116,-0.025787,0.156653,0.112991,-0.068387,-0.017055,0.011229
2022-04-01,-0.028395,-0.093948,-0.041061,-0.109332,-0.130964,-0.007755,-0.036356,-0.085459,-0.056298,-0.031300
2022-05-01,-0.048941,0.011863,0.039015,-0.003367,0.051850,0.014069,0.021817,0.033525,0.036282,0.035155
2022-06-01,-0.088328,0.039289,-0.075004,-0.069725,-0.121062,-0.141624,-0.057863,-0.090486,-0.000257,-0.124403


## Find the optimal allocation

Asset groups are left as exercise to the reader.

In [5]:
import opvious

client = opvious.Client.default()

response = await client.run_solve(
    specification=opvious.RemoteSpecification.example('portfolio-selection'),
    parameters={
        'assetCovariance': returns_df.cov().stack(),
        'expectedAssetReturn': returns_df.mean(),
        'minimumReturn': 0.005,
        'assetGroup': {},
        'minGroupAllocation': {},
    },
    assert_feasible=True,
)

20:51:59 INFO Validated inputs. [parameters=111]
20:51:59 INFO Solving problem... [columns=10, rows=2]
20:51:59 INFO Solve completed with status OPTIMAL. [objective=0.0012731654496557063]


In [6]:
allocation = response.outputs.variable('allocation')
allocation.reset_index(names=['ticker']).join(
    returns_df.agg(['mean', 'var']).T,
    on='ticker',
    validate='one_to_one'
).sort_values(by=['value'], ascending=False)

,ticker,value,dual_value,mean,var
8,ATVI,0.486597,0.000000,0.005283,0.002387
6,ADP,0.195967,0.000000,0.007451,0.004878
1,ABBV,0.176768,0.000000,0.004639,0.004765
5,ADM,0.066385,0.000000,0.006048,0.008350
2,ABT,0.053921,0.000000,-0.006146,0.003164
7,AOS,0.020361,0.000000,0.003891,0.011149
0,AAP,0.000000,0.006336,-0.054546,0.019300
3,ACN,0.000000,0.000542,-0.003617,0.006525
4,ADBE,0.000000,0.001086,0.003217,0.017312
9,MMM,0.000000,0.003104,-0.022114,0.006804
